In [ ]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from keras.models import load_model
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import threading
from tensorflow.keras.models import model_from_json

from age_gender_prediction import predict_age, age_net
with open('model_arch.json') as json_file:
    loaded_model= json_file.read()

emotion_model = model_from_json(loaded_model)

emotion_model.load_weights("model.weights.h5")

emotion_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print("Model loaded and compiled successfull")  

AGE_LABELS = ["0-2","4-6","8-12","13-17","18-24","25-32","38-43","48-53","60-100"]
EMOTIONS = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]


log_file = "theatre_log.csv"
df = pd.DataFrame(columns=["Age","Emotion","EntryTime"])
df.to_csv(log_file, index=False)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


root = tk.Tk()
root.title("Movie Theatre Age & Emotion Detection")

lmain = tk.Label(root)
lmain.pack()

cap = cv2.VideoCapture(0)
running = False

def detect_faces():
    global running
    while running:
        ret, frame = cap.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x,y,w,h) in faces:
            face = frame[y:y+h, x:x+w]

            # ---- Age Prediction ----
            try:
                face_resized = cv2.resize(face, (64,64))
                face_norm = face_resized / 255.0
                face_input = np.expand_dims(face_norm, axis=0)
                age_pred = predict_age(face_input,age_net)
                age_bucket = AGE_LABELS[np.argmax(age_pred)]
                approx_age = int(age_bucket.split("-")[0])
            except:
                approx_age = 25
                age_bucket = "25-32"

            # ---- Decision ----
            if approx_age < 13 or approx_age > 60:
                color = (0,0,255)
                label = f"Age:{approx_age} Not allowed"
                emotion = "N/A"
            else:
                roi_gray = gray[y:y+h, x:x+w]
                roi_gray = cv2.resize(roi_gray, (48,48)) / 255.0
                roi_gray = np.expand_dims(np.expand_dims(roi_gray, -1), 0)
                emotion_pred = emotion_model.predict(roi_gray)
                emotion = EMOTIONS[np.argmax(emotion_pred)]
                color = (0,255,0)
                label = f"Age:{approx_age} Emotion:{emotion}"

            cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
            cv2.putText(frame, label, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # ---- Log Data ----
            entry_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log = pd.DataFrame([[approx_age, emotion, entry_time]],
                               columns=["Age","Emotion","EntryTime"])
            log.to_csv(log_file, mode='a', header=False, index=False)

        
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)

def start_detection():
    global running
    if not running:
        running = True
        threading.Thread(target=detect_faces, daemon=True).start()

def stop_detection():
    global running
    running = False
    messagebox.showinfo("Stopped", "Detection stopped. Log saved to theatre_log.csv")

btn_frame = tk.Frame(root)
btn_frame.pack()

start_btn = tk.Button(btn_frame, text="Start Detection", command=start_detection, bg="green", fg="white")
start_btn.pack(side=tk.LEFT, padx=10, pady=10)

stop_btn = tk.Button(btn_frame, text="Stop Detection", command=stop_detection, bg="red", fg="white")
stop_btn.pack(side=tk.LEFT, padx=10, pady=10)

root.protocol("WM_DELETE_WINDOW", stop_detection)
root.mainloop()

cap.release()
cv2.destroyAllWindows()


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 20 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded and compiled successfull
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


Exception in thread Thread-4 (detect_faces):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_14276\2617342005.py", line 96, in detect_faces
  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\PIL\ImageTk.py", line 129, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "c:\Program Files\Python310\lib\tkinter\__init__.py", line 4103, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "c:\Program Files\Python310\lib\tkinter\__init__.py", line 4048, in __init__
    self.tk.call(('image', 'create', imgtype, name,) + options)
RuntimeError: main